In [1]:
import pandas as pd

In [2]:
df_bruto = pd.read_csv('dataset_bruto.csv')
df = pd.read_csv('dataset.csv')

In [3]:
import pandas as pd
import plotly.graph_objects as go

def plot_correlation_heatmap(df):
    # matriz de correlação
    correlacoes = df.corr(numeric_only=True)
    variaveis = correlacoes.columns.tolist()

    # Heatmap interativo
    fig = go.Figure(data=go.Heatmap(
        z=correlacoes.values,
        x=variaveis,
        y=variaveis,
        colorscale='RdBu',
        zmin=-1,
        zmax=1,
        colorbar=dict(title="Correlação"),
        hovertemplate=(
            "Variável X: %{x}<br>"
            "Variável Y: %{y}<br>"
            "Correlação: %{z:.2f}<extra></extra>"
        )
    ))

    # Ajustar layout
    fig.update_layout(
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        title="Mapa de Calor das Correlações (Interativo)",
        width=800,
        height=800
    )

    fig.update_yaxes(autorange='reversed')

    fig.show()


In [10]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def plot_correlacoes_interativas(df):
    corr = df.corr(numeric_only=True)
    np.fill_diagonal(corr.values, np.nan)
    variaveis = corr.columns.tolist()

    # Slider de limiar entre -1.0 e 1.0
    slider = widgets.FloatSlider(
        value=0.7,
        min=-1.0,
        max=1.0,
        step=0.05,
        description='Limiar:',
        continuous_update=False,
        readout_format='.2f',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Função de atualização
    def atualizar_grafico(limiar):
        if limiar >= 0:
            mask = corr >= limiar
        else:
            mask = corr <= limiar

        filtered_corr = corr.where(mask)

        fig = go.Figure(data=go.Heatmap(
            z=filtered_corr.values,
            x=variaveis,
            y=variaveis,
            colorscale='RdBu',
            zmin=-1,
            zmax=1,
            zmid=0,
            colorbar=dict(title="Correlação"),
            hovertemplate="Variável X: %{x}<br>Variável Y: %{y}<br>Correlação: %{z:.2f}<extra></extra>"
        ))

        fig.update_layout(
            title=f"Mapa de Correlações | Limiar: {limiar}",
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            width=600,
            height=600
        )
        fig.update_yaxes(autorange='reversed')
        fig.show()

    # Exibir
    widgets.interact(atualizar_grafico, limiar=slider)


In [ ]:
plot_correlacoes_interativas(df_bruto)

interactive(children=(FloatSlider(value=0.7, continuous_update=False, description='Limiar:', layout=Layout(wid…

In [6]:
plot_correlacoes_interativas(df)

interactive(children=(FloatSlider(value=0.7, continuous_update=False, description='Limiar:', layout=Layout(wid…

In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import pandas as pd

def kmeans_com_plot_sem_pca(df, features, x_axis, y_axis, n_clusters=3):
    for col in features + [x_axis, y_axis]:
        if col not in df.columns:
            raise ValueError(f"Coluna '{col}' não encontrada no DataFrame.")

    dados = df[features].dropna()
    scaler = StandardScaler()
    dados_normalizados = scaler.fit_transform(dados)

    # KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(dados_normalizados)

    # DataFrame para visualização
    df_plot = df.loc[dados.index].copy()
    df_plot['cluster'] = clusters

    fig = px.scatter(
        df_plot,
        x=x_axis,
        y=y_axis,
        color=df_plot['cluster'].astype(str),
        title=f"KMeans com {n_clusters} clusters",
        labels={'color': 'Cluster'},
        hover_data=features
    )

    fig.update_layout(width=800, height=600)
    fig.show()

    features.append('local')
    df_resultado = df[features]
    df_resultado.loc[dados.index, 'cluster'] = clusters

    print(df_resultado['cluster'].value_counts().sort_index())

    return df_resultado


In [8]:
features = ['num_pedidos_residencia', 'num_crimes_municipios']
print(df.columns)
df_clusterizado = kmeans_com_plot_sem_pca(df, features, x_axis='num_pedidos_residencia', y_axis='num_crimes_municipios', n_clusters=3)
df_clusterizado

Index(['local', 'ensino_secundario_completo', 'total_medio_pensoes',
       'beneficiarios_rsi_por_100hab', 'taxa_criminalidade',
       'retencao_desistencia_ensino_basico', 'num_pedidos_residencia',
       'empregada_por_conta_outrem_com_ensino_superior',
       'taxa_bruta_escolarizacao_ensino_secundario', 'ganho_mensal',
       'taxa_crescimento', 'Duracao_sub_desemprego', 'saldo_migratorio',
       'votos_por_partido_ou_coligacao', 'populacao_ativa', 'saldo_natural',
       'residente_empregada_estudante', 'beneficiarios_abono_familia',
       'valor_rsi', 'num_estabelecimentos_ensino_nao_superior',
       'num_crimes_municipios', 'Idade_População', 'taxa_de_abstenção',
       'poder_de_compra', 'pop_empregada_outrem_escalão', 'taxa_emprego',
       'credito_habitacao_permanente', 'votos_nulos_assembleias_municipais',
       'pop_inscrita_eleicoes_assembleias_municipais', 'Índice De Renovação',
       'Densidade_População', 'Num_Beneficiarios_sub_desemprego',
       'rendimento_br

/Users/mariainesfernandes/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



cluster
0.0    242
1.0     13
2.0     56
Name: count, dtype: int64


/var/folders/0q/ysq7ngqx1p5d0wj8y5tm8mr80000gn/T/ipykernel_12721/1211900184.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,num_pedidos_residencia,num_crimes_municipios,local,cluster
0,0.0,0.0,Abrantes,0.0
1,0.0,0.0,Aguiar da Beira,0.0
2,0.0,0.0,Alandroal,0.0
3,0.0,0.0,Albergaria-a-Velha,0.0
4,0.0,2.0,Albufeira,2.0
...,...,...,...,...
311,0.0,0.0,Vouzela,0.0
312,0.0,1.0,Águeda,2.0
313,0.0,1.0,Évora,2.0
314,0.0,1.0,Ílhavo,2.0


In [9]:
df.columns

Index(['local', 'ensino_secundario_completo', 'total_medio_pensoes',
       'beneficiarios_rsi_por_100hab', 'taxa_criminalidade',
       'retencao_desistencia_ensino_basico', 'num_pedidos_residencia',
       'empregada_por_conta_outrem_com_ensino_superior',
       'taxa_bruta_escolarizacao_ensino_secundario', 'ganho_mensal',
       'taxa_crescimento', 'Duracao_sub_desemprego', 'saldo_migratorio',
       'votos_por_partido_ou_coligacao', 'populacao_ativa', 'saldo_natural',
       'residente_empregada_estudante', 'beneficiarios_abono_familia',
       'valor_rsi', 'num_estabelecimentos_ensino_nao_superior',
       'num_crimes_municipios', 'Idade_População', 'taxa_de_abstenção',
       'poder_de_compra', 'pop_empregada_outrem_escalão', 'taxa_emprego',
       'credito_habitacao_permanente', 'votos_nulos_assembleias_municipais',
       'pop_inscrita_eleicoes_assembleias_municipais', 'Índice De Renovação',
       'Densidade_População', 'Num_Beneficiarios_sub_desemprego',
       'rendimento_br